# Load Horizons Data into KeplerDB

In [1]:
# Core
import numpy as np
import pandas as pd

# Utility
import re
import glob
import time

# Database
import sqlalchemy
from sqlalchemy import create_engine

In [2]:
# MSE imports
import kepler_sieve
from horizons_files import hrzn_txt2csv, hrzn_csv2df, hrzn_df2db, hrzn_txt2db, hrzn_csv2db, hrzn_load
from asteroid_element import load_ast_elt, load_data_impl, convert_data
from astro_utils import anomaly_M2E, anomaly_E2f
import db_config

Converged with error 8.88e-16 after 25 iterations.


## Load all the Horizons CSV Files into JPL.HorizonsImport

In [ ]:
# hrzn_load()

## Testing - Load CSV, Display as DataFrame

In [ ]:
fname_txt = '../data/jpl/horizons/planets/daily/010_sun.txt'
# fname_txt = '../data/jpl/horizons/planets/daily/001_mercury_barycenter.txt'
# fname_txt = '../data/jpl/horizons/moons/daily/301_moon.txt'
# fname_txt = '../data/jpl/horizons/asteroids/weekly/ast_0001.txt'

In [ ]:
fname_csv = hrzn_txt2csv(fname_txt)

In [ ]:
fname_csv

In [ ]:
df = hrzn_csv2df(fname_csv)

In [ ]:
pd.set_option('display.max_rows', 10)

In [ ]:
df

## Testing - Populate Database from CSV

In [ ]:
connection_str = f'mysql+pymysql://{db_config.username}:{db_config.password}@{db_config.hostname}/JPL'

In [ ]:
engine = create_engine(connection_str)

In [ ]:
engine.table_names()

In [ ]:
# hrzn_txt2db(fname_txt)

## Load the Asteroid Orbital Elements from JPL

In [3]:
df = load_ast_elt()

In [4]:
df

,Num,Name,epoch,a,e,inc,Omega,omega,M,H,G,Ref,f,P,n,long,theta,pomega,T_peri,row_num
Num,,,,,,,,,,,,,,,,,,,,
1,1,Ceres,58600.0,2.769165,0.076009,0.184901,1.401596,1.284522,1.350398,3.40,0.12,JPL 47,1.501306,1683.145658,0.003733,4.036516,4.187424,2.686118,-361.745861,0
2,2,Pallas,59000.0,2.773841,0.229972,0.607949,3.019851,5.414053,2.530303,4.20,0.11,JPL 37,2.742191,1687.410960,0.003724,4.681022,4.892910,2.150719,-679.537622,1
3,3,Juno,59000.0,2.668285,0.256936,0.226736,2.964468,4.329572,2.189260,5.33,0.32,JPL 112,2.535135,1592.013779,0.003947,3.200114,3.545989,1.010854,-554.707814,2
4,4,Vesta,58600.0,2.361418,0.088721,0.124647,1.811840,2.630709,1.673106,3.00,0.32,JPL 35,1.846768,1325.432768,0.004740,6.115656,0.006132,4.442550,-352.940426,3
5,5,Astraea,59000.0,2.574037,0.190913,0.093679,2.470881,6.259596,0.311477,6.90,0.15,JPL 114,0.461775,1508.414438,0.004165,2.758769,2.909067,2.447291,-74.776891,4
6,6,Hebe,59000.0,2.424533,0.203219,0.257255,2.419784,4.184187,3.328107,5.80,0.24,JPL 89,3.267907,1378.924524,0.004557,3.648893,3.588693,0.320786,648.529285,5
7,7,Iris,59000.0,2.387375,0.230145,0.096370,4.530245,2.534245,4.318395,5.60,0.15,JPL 110,3.944513,1347.347036,0.004663,5.099700,4.725818,0.781305,421.323609,6
8,8,Flora,59000.0,2.201415,0.155833,0.102784,1.935160,4.982198,5.503338,6.50,0.28,JPL 118,5.251935,1193.029603,0.005267,6.137511,5.886108,0.634173,148.074814,7
9,9,Metis,59000.0,2.386189,0.123300,0.097328,1.202697,0.110607,0.417347,6.30,0.17,JPL 116,0.533303,1346.343243,0.004667,1.730651,1.846607,1.313304,-89.427873,8


In [5]:
M = df.M.values
e = df.e.values

In [6]:
e.shape

(958724,)

In [7]:
E = anomaly_M2E(M=M, e=e)

Converged with error 8.88e-16 after 7 iterations.


In [8]:
tau = 2.0 * np.pi

In [9]:
f_mse = anomaly_E2f(E, e)

In [10]:
f_reb = df.f.values

In [11]:
f_err = np.abs(f_mse - f_reb)

In [12]:
np.max(np.sin(f_err))

6.650369144267643e-10

In [ ]:
# Load data from JPL asteroids file
df_in = load_data_impl()
# Convert data to rebound format
ast_elt = convert_data(df_in=df_in)

In [ ]:
ast_elt

In [ ]:
tau

In [ ]:
# Number of asteroids
N: int = len(ast_elt)

# Initialize empty arrays for computed orbital elements
f = np.zeros(N)
P = np.zeros(N)
mean_motion = np.zeros(N)
long = np.zeros(N)
theta = np.zeros(N)
pomega = np.zeros(N)

In [ ]:
np.min(M)

In [ ]:
np.max(M)